In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math

import torch
import torch.nn as nn
from torch.optim import RMSprop

from torch.utils.data import DataLoader, TensorDataset

In [13]:
data = pd.read_csv('../data/jhu-covid-cases.csv')
data = data[data.Province_State == 'West Virginia']
data = data[data.columns[11:]]
print(data.columns[:5])

death = pd.read_csv('../data/jhu-covid-deaths.csv')
death = death[death.Province_State == 'West Virginia']
death = death[death.columns[12:]]
print(death.columns[:5]) # make sure the dates match

data = np.array(data.sum(axis = 0)) #.reshape(-1, 1)
data = np.convolve(data, np.ones(7) / 7, 'valid')
data = torch.tensor(data)
Itilde = torch.log(1 + data[20:] - data[:-20]).reshape(1, -1)

death = torch.tensor(death.sum(axis = 0))
Dt = torch.tensor(np.convolve(death, np.ones(7) / 7, 'valid'))
Dtilde = torch.log(1 + Dt[:-20]).reshape(1, -1)

N = 1_775_000 
Ntilde = math.log(N)

Stilde = torch.log(N - torch.exp(Itilde) - torch.exp(Dtilde))

first_index = torch.arange(1117)[Itilde[0,:] > 4.6][0].item() # first index above 100 cases 

Stilde = Stilde[:,first_index:].T
Itilde = Itilde[:,first_index:].T
Dtilde = Dtilde[:,first_index:].T

Y = torch.hstack((Stilde, Itilde, Dtilde))
T = torch.arange(1073).reshape(-1, 1) * 1.

print(T.shape, Y.shape) # make sure shapes match 

Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20'], dtype='object')
Index(['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20'], dtype='object')
torch.Size([1073, 1]) torch.Size([1073, 3])


In [14]:
class Greek(nn.Module): # 1 to 1 (time to positive value)
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(1, 16)
        self.lin2 = nn.Linear(16, 16)
        self.lin3 = nn.Linear(16, 16)
        self.lin4 = nn.Linear(16, 1)

        self.act = nn.GELU()

        #self.param = nn.Parameter(torch.randn(1078, 1))

    #"""
    def forward(self, x):
        x = x / 1000 # ich hab kein lust das zu preprocessieren :P
        x = self.lin1(x)
        x = self.act(x)
        x = self.lin2(x)
        x = self.act(x)
        x = self.lin3(x)
        x = self.act(x)
        x = self.lin4(x)
        return 1 + torch.sin(x)

    #"""
    #def forward(self, t):
    #    print(t)
    #    return torch.exp(self.param[t])

class Curves(nn.Module): # put all 3 in 1
    def __init__(self, T = 1078):
        super().__init__()
        self.lin1 = nn.Linear(1, 48)
        self.lin2 = nn.Linear(48, 48)
        self.lin3 = nn.Linear(48, 48)
        self.lin4 = nn.Linear(48, 48)
        self.lin5 = nn.Linear(48, 3)

        self.act = nn.GELU()

        self.gamma = Greek()
        self.beta = Greek()
        self.mu = Greek()

    def forward(self, x): # takes time and produces S, I, T estimates
        x = x / 1000 # again to avoid preprocessing
        x = self.lin1(x)
        x = self.act(x)
        x = self.lin2(x)
        x = self.act(x)
        x = self.lin3(x)
        x = self.act(x)
        x = self.lin4(x)
        x = self.act(x)
        x = self.lin5(x)

        return Ntilde * x # so the model only has to predict something smaller

    def get_derivative(self, t):
        t = t / 1000
        t.requires_grad = True
        
        x = self.forward(t)

        grad_s = torch.autograd.grad(x[:,0].sum(), t, create_graph=True)[0]
        grad_i = torch.autograd.grad(x[:,1].sum(), t, create_graph=True)[0]
        grad_d = torch.autograd.grad(x[:,2].sum(), t, create_graph=True)[0]
        
        return grad_s, grad_i, grad_d


In [ ]:
NN = Curves()
optimizer = RMSprop(NN.parameters(), lr = 1e-3)

EPOCHS = 800
dataset = DataLoader(TensorDataset(T, Y), batch_size = 25, shuffle = True)
losses = []

for i in range(EPOCHS):

    total_loss = 0.

    for t, y in dataset:
        optimizer.zero_grad()
        
        y_hat = NN(t)

        ## assemble losses
        ### distance from the model output to the actual curves
        curve_loss = torch.sum((y - y_hat)**2)
        
        ### pde losses
        dsdt, didt, dddt = NN.get_derivative(t)

        mu_t = NN.mu(t)
        beta_t = NN.beta(t)
        gamma_t = NN.gamma(t)

        dsdt_hat = gamma_t * torch.exp(y_hat[:,2] - y_hat[:,0]).reshape(-1, 1) - beta_t * torch.exp(y_hat[:,1] - Ntilde).reshape(-1, 1)
        dddt_hat = mu_t * torch.exp(y_hat[:,1] - y_hat[:,2]).reshape(-1, 1)
        didt_hat = beta_t * torch.exp(y_hat[:,0] - Ntilde).reshape(-1, 1) - \
                    mu_t * torch.exp(y_hat[:,2] - y_hat[:,1]).reshape(-1, 1) - gamma_t

        dsdt_loss = torch.sum((dsdt - dsdt_hat)**2)
        dddt_loss = torch.sum((dddt - dddt_hat)**2)
        didt_loss = torch.sum((didt - didt_hat)**2)

        pde_loss = dddt_loss + dsdt_loss + didt_loss

        ### boundary loss
        pde_bound = torch.sum((dsdt + didt + dddt)**2)
        curve_bound = 0. #torch.sum((Ntilde - torch.log(torch.exp(y_hat).sum(axis = 1)))**2)

        boundary_loss = pde_bound + curve_bound
        
        ### total loss assemble
        l = 100 * pde_loss + curve_loss + (10 * boundary_loss)
        
        l.backward()
        optimizer.step()

        total_loss += l.item() #.detach()

    losses.append(total_loss)

    if i % 9 == 0:
        print('=== Epoch', i, '| PDE:', round(100 * pde_loss.item(), 4), '| Curve:', round(curve_loss.item(), 4), '| Boundary:', round(10 * boundary_loss.item(), 4),  '===')
    


=== Epoch 0 | PDE: 2649.9073 | Curve: 6556.3828 | Boundary: 0.0 ===
=== Epoch 9 | PDE: 2.7326 | Curve: 70.7307 | Boundary: 0.0127 ===
=== Epoch 18 | PDE: 0.1051 | Curve: 36.7107 | Boundary: 0.0501 ===
=== Epoch 27 | PDE: 0.161 | Curve: 46.049 | Boundary: 0.1699 ===
=== Epoch 36 | PDE: 0.3928 | Curve: 24.6784 | Boundary: 0.3414 ===
=== Epoch 45 | PDE: 1.8514 | Curve: 24.8161 | Boundary: 0.5841 ===
=== Epoch 54 | PDE: 1.2099 | Curve: 39.5728 | Boundary: 0.7393 ===
=== Epoch 63 | PDE: 0.3523 | Curve: 34.8593 | Boundary: 0.6932 ===
=== Epoch 72 | PDE: 1.4945 | Curve: 22.5716 | Boundary: 0.732 ===
=== Epoch 81 | PDE: 0.1353 | Curve: 20.3665 | Boundary: 0.6553 ===
=== Epoch 90 | PDE: 0.2909 | Curve: 12.5315 | Boundary: 0.6289 ===
=== Epoch 99 | PDE: 0.0625 | Curve: 22.1239 | Boundary: 0.5582 ===
=== Epoch 108 | PDE: 0.2144 | Curve: 18.4213 | Boundary: 0.4698 ===
=== Epoch 117 | PDE: 0.1342 | Curve: 25.3807 | Boundary: 0.402 ===
=== Epoch 126 | PDE: 0.4681 | Curve: 16.2893 | Boundary: 0.236 =

In [ ]:
plt.plot(losses[20:])

In [ ]:
with torch.no_grad():
    plt.plot(NN(T)[:,2])
    plt.plot(Dtilde)

In [ ]:
with torch.no_grad():
    plt.plot(NN(T)[:,1])
    plt.plot(Itilde)

In [ ]:
with torch.no_grad():
    plt.plot(NN.beta(T)[:,0])

In [ ]:
with torch.no_grad():
    plt.plot(NN.gamma(T)[:,0])

In [ ]:
with torch.no_grad():
    plt.plot(NN.mu(T)[:,0])